#TP4 NLP: Criação do baseline



----
Aluno PPGI: Igor de Souza Lima

In [ ]:
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3
    !pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.6.

In [ ]:
import json
import sqlite3
import os
import torch
from unsloth import FastLanguageModel

# --- 1. CONFIGURAÇÃO DOS CAMINHOS E PARÂMETROS ---
DEV_FILE = "/dados/dev.json"
PROMPT_TEMPLATE_FILE = "/dados/prompt_template.txt"
DATABASE_DIR = "/dados/"  # Diretório que contém a pasta 'database'
SEED = 42

TEMPERATURE = 0.1
TOP_P = 0.95

# --- 2. FUNÇÃO OTIMIZADA E CORRIGIDA PARA CHAMAR O MODELO ---
def chamar_meu_modelo(model, tokenizer, prompt: str) -> str:
    """
    Recebe o modelo e o tokenizador já carregados e o prompt para gerar a consulta SQL.
    Esta versão foi corrigida para extrair APENAS a resposta do assistente.
    """
    messages = [
        {"role": "user", "content": prompt},
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=256,
        use_cache=True,
        do_sample=True,
        temperature=TEMPERATURE,
        top_p=TOP_P,
    )

    # Decodifica a saída completa
    full_response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # **INÍCIO DA LÓGICA DE CORREÇÃO**
    try:
        # 1. Isola a parte da resposta do assistente usando o separador do Llama 3
        assistant_tag = "assistant\n\n" # O template do Llama 3 insere isso após o header.
        assistant_part = full_response.split(assistant_tag)[-1].strip()

        # 2. Procura por um bloco de código SQL dentro da resposta
        sql_block_start = assistant_part.find("```sql")
        if sql_block_start != -1:
            # Se encontrou o início do bloco, procura o fim
            sql_block_end = assistant_part.rfind("```")
            if sql_block_end > sql_block_start:
                # Extrai o conteúdo entre as tags ```sql e ```
                sql_code = assistant_part[sql_block_start + len("```sql") : sql_block_end].strip()
                return sql_code

        # 3. Se não houver bloco de código, retorna a resposta do assistente como está
        # (pode ser que o modelo tenha retornado apenas a SQL diretamente)
        return assistant_part

    except Exception:
        # Se qualquer parte da extração falhar, retorna a resposta completa como último recurso
        return full_response
    # **FIM DA LÓGICA DE CORREÇÃO**

# --- 3. LÓGICA PRINCIPAL DA AVALIAÇÃO (sem alterações) ---
def avaliar_baseline(model, tokenizer):
    try:
        with open(DEV_FILE, 'r', encoding='utf-8') as f:
            dev_data = json.load(f)
        with open(PROMPT_TEMPLATE_FILE, 'r', encoding='utf-8') as f:
            prompt_template = f.read()
    except FileNotFoundError as e:
        print(f"Erro: Arquivo não encontrado - {e}. Verifique os caminhos.")
        return

    sucessos = 0
    falhas = 0
    total = len(dev_data)

    for i, item in enumerate(dev_data):
        pergunta = item['question']
        db_id = item['db_id']
        sql_correta = item['query']

        print(f"\n--- AVALIANDO ITEM {i+1}/{total} | BANCO DE DADOS: {db_id} ---")
        print(f"PERGUNTA: {pergunta}")

        prompt_final = prompt_template.replace('{your_new_question_here}', pergunta)
        sql_gerada = chamar_meu_modelo(model, tokenizer, prompt_final)

        # Remove ponto e vírgula do final, se houver, para evitar erros no SQLite
        if sql_gerada.endswith(';'):
            sql_gerada = sql_gerada[:-1]

        print(f"SQL GERADA: {sql_gerada}")
        print(f"SQL CORRETA: {sql_correta}")

        db_path = os.path.join(DATABASE_DIR, 'database', db_id, f'{db_id}.sqlite')

        if not os.path.exists(db_path):
            print(f"  -> FALHA (Erro: Banco de dados não encontrado em '{db_path}')")
            falhas += 1
            continue

        try:
            conn = sqlite3.connect(db_path)
            cursor_gerada = conn.cursor()
            cursor_correta = conn.cursor()

            cursor_gerada.execute(sql_gerada)
            resultado_gerado = cursor_gerada.fetchall()

            cursor_correta.execute(sql_correta)
            resultado_correto = cursor_correta.fetchall()

            if resultado_gerado == resultado_correto:
                print("  -> SUCESSO (Resultados idênticos)")
                sucessos += 1
            else:
                print("  -> FALHA (Resultado diferente do esperado)")
                falhas += 1

        except sqlite3.Error as e:
            print(f"  -> FALHA (Erro de execução SQL: {e})")
            falhas += 1

        finally:
            if 'conn' in locals():
                conn.close()

    # --- 4. EXIBIR RESULTADOS FINAIS ---
    print("\n\n--- AVALIAÇÃO DO BASELINE CONCLUÍDA ---")
    print(f"Total de Perguntas Avaliadas: {total}")
    print(f"Sucessos: {sucessos}")
    print(f"Falhas: {falhas}")

    taxa_de_sucesso = (sucessos / total) * 100 if total > 0 else 0
    print(f"Taxa de Sucesso (Execution Accuracy): {taxa_de_sucesso:.2f}%")


if __name__ == '__main__':
    torch.manual_seed(SEED)

    print("Carregando o modelo e o tokenizador...")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
        max_seq_length=2048,
        dtype=None,
        load_in_4bit=True,
    )

    FastLanguageModel.for_inference(model)
    print("Modelo carregado com sucesso.")

    avaliar_baseline(model, tokenizer)

A saída de streaming foi truncada nas últimas 5000 linhas.
FROM tv_series AS T1
ORDER BY T1.rating DESC
LIMIT 3
SQL CORRETA: SELECT Episode ,  Rating FROM TV_series ORDER BY Rating DESC LIMIT 3;
  -> SUCESSO (Resultados idênticos)

--- AVALIANDO ITEM 617/1034 | BANCO DE DADOS: tvshow ---
PERGUNTA: What are 3 most highly rated episodes in the TV series table and what were those ratings?
SQL GERADA: SELECT title, rating
FROM tv_series
ORDER BY rating DESC
LIMIT 3
SQL CORRETA: SELECT Episode ,  Rating FROM TV_series ORDER BY Rating DESC LIMIT 3;
  -> FALHA (Erro de execução SQL: no such column: title)

--- AVALIANDO ITEM 618/1034 | BANCO DE DADOS: tvshow ---
PERGUNTA: What is minimum and maximum share of TV series?
SQL GERADA: SELECT 
    (min(tv_series.num_episodes) / (SELECT COUNT(*) FROM tv_series)) AS min_share,
    (max(tv_series.num_episodes) / (SELECT COUNT(*) FROM tv_series)) AS max_share
FROM 
    tv_series
SQL CORRETA: SELECT max(SHARE) , min(SHARE) FROM TV_series;
  -> FALHA (E